<a href="https://colab.research.google.com/github/SamuelRoblesArce/Modelo_Advertising/blob/main/ModeloDeAdvertising.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Advertising

El objetivo es usar regresión multivariada para entender como los gastos en anuncios impactan las ventas (variable a predecir es **sales**).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
url = "https://raw.githubusercontent.com/eduardool/MaestriaCienciaDatos/main/datasets/advertising_regresion.csv"

In [ ]:
datos = pd.read_csv(url)

In [ ]:
datos.head()

,Unnamed: 0,TV,radio,newspaper,sales
0,1,230.1,37.8,69.2,22.1
1,2,44.5,39.3,45.1,10.4
2,3,17.2,45.9,69.3,9.3
3,4,151.5,41.3,58.5,18.5
4,5,180.8,10.8,58.4,12.9


Separar $X$ y $y$

In [ ]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newspaper   200 non-null    float64
 4   sales       200 non-null    float64
dtypes: float64(4), int64(1)
memory usage: 7.9 KB


In [ ]:
datos = datos.iloc[:, 1:]
datos

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [ ]:
X,y = datos.values[:, :-1], datos.values[:, -1]

X=X.astype('float32')
y=y.astype('float32')

Separar conjunto de entrenamiento (80\%) y prueba (20\%), random_state = 100

# Regresión Linear

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=100)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(160, 3) (40, 3) (160,) (40,)


Calcular y realizar la escala de variables sobre el conjunto entrenamiento.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lm = LinearRegression()

In [ ]:
lm.fit(X_test,y_test) # aprendizaje ML

LinearRegression()

In [ ]:
from sklearn.model_selection import cross_val_score
scores_lm = cross_val_score(lm, X_train,y_train, cv=5, scoring='neg_mean_squared_error') # validación cruzada de 5 folds
print("RMSE promedio de Linear Regression: %0.2f (+/- %0.2f)" % (np.sqrt(-scores_lm.mean()), scores_lm.std()))

RMSE promedio de Linear Regression: 1.86 (+/- 1.02)


# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor
clasificador_rf = RandomForestRegressor(random_state=100)

In [ ]:
scores_rf = cross_val_score(clasificador_rf, X_train, y_train, cv=5, scoring='neg_mean_squared_error')

print("RMSE promedio de Random Forest: %0.2f (+/- %0.2f)" % (np.sqrt(-scores_rf.mean()), scores_rf.std()))


RMSE promedio de Random Forest: 0.93 (+/- 0.41)


Usar validación cruzada (5-folds) para comparar los algoritmos de regresión lineal multivariada y Random forest. En ambos algoritmos usar un random_state = 100. Cual algoritmo es mejor?

In [ ]:
#El mejor es Random Forest que tiene el menor RMSE.

Realizar el afinamiento del algoritmo que tuvo mejor desempeño (usar GridSearchCV).


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [1,2,3]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [1, 2, 3]},
  ]

reg_forest = RandomForestRegressor(random_state=100)

grid_search = GridSearchCV(reg_forest, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=100),
             param_grid=[{'max_features': [1, 2, 3],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [1, 2, 3],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [ ]:
grid_search.best_params_

{'max_features': 3, 'n_estimators': 30}

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)
# Combinación de resultados obtenidos durante la búsqueda

2.1983784855385053 {'max_features': 1, 'n_estimators': 3}
1.718007204530469 {'max_features': 1, 'n_estimators': 10}
1.7344733707957445 {'max_features': 1, 'n_estimators': 30}
1.3125965439734444 {'max_features': 2, 'n_estimators': 3}
1.1431396255687052 {'max_features': 2, 'n_estimators': 10}
1.0160146226650446 {'max_features': 2, 'n_estimators': 30}
1.1054662755782783 {'max_features': 3, 'n_estimators': 3}
1.02363508615145 {'max_features': 3, 'n_estimators': 10}
0.9377274684155964 {'max_features': 3, 'n_estimators': 30}
2.377831698612854 {'bootstrap': False, 'max_features': 1, 'n_estimators': 3}
1.8485651805391576 {'bootstrap': False, 'max_features': 1, 'n_estimators': 10}
1.5229973378354922 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
1.07071853921067 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
1.3182638395543893 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
1.3119477778751114 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}


Usando Randomized

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=3),
    }

reg_forest = RandomForestRegressor(random_state=100)
rnd_search = RandomizedSearchCV(reg_forest, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=100)
rnd_search.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(random_state=100),
                   param_distributions={'max_features': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7922b30ff400>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7922b30fe6b0>},
                   random_state=100, scoring='neg_mean_squared_error')

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

1.7298420783722641 {'max_features': 1, 'n_estimators': 25}
0.9886679355939294 {'max_features': 2, 'n_estimators': 104}
0.9923792955585414 {'max_features': 2, 'n_estimators': 80}
1.8236635610488057 {'max_features': 1, 'n_estimators': 139}
1.794429463878243 {'max_features': 1, 'n_estimators': 181}
1.7680059891092126 {'max_features': 1, 'n_estimators': 54}
1.645384277816529 {'max_features': 1, 'n_estimators': 15}
1.7298420783722641 {'max_features': 1, 'n_estimators': 25}
0.9907184183799962 {'max_features': 2, 'n_estimators': 108}
1.8035725997301406 {'max_features': 1, 'n_estimators': 59}


In [ ]:
modelo_final = rnd_search.best_estimator_
modelo_final

RandomForestRegressor(max_features=2, n_estimators=104, random_state=100)

Reportar el RMSE del algoritmo afinado sobre el conjunto de prueba (no olvidarse de antes aplicar la escala calculada sobre el conjunto de entrenamiento).

El RMSE es 0.9377274684155964, usando RandomForest, con 3 features, 30 n_estimators.